In [2]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None

- Прочитать json файлы
- Филтрануть du_ref_strategy по самой поздней дате загрузки данных (added_at_ts)
- Заджоинить с du_profitability_strategy
- Слайсинг с начала этого года и по столбцам: 'date', 'link', 'name', 'code_out', 'cur_name', 'previousaum', 'currentaum', 'cashflow'
- Убрать строки где текущий и предыдущий СЧА нулевой (чтобы не возникала беск. дох-ть)
- Сортировка по столбцам: 'name', 'code_out', 'date'

In [11]:
profit = pd.read_json('du_profitability_strategy.json', convert_dates=['added_at_ts'])
rename_col = {
    'strategy': 'link'
}
profit = profit.rename(rename_col, axis=1)
strategy_info = pd.read_json('du_ref_strategy.json', convert_dates=['added_at_ts'])

strategy_info = strategy_info.loc[
    strategy_info['added_at_ts'] == strategy_info['added_at_ts'].max()
]
df = pd.merge(
    profit, strategy_info, 'left', 'link'
)
cols = [
    'date', 'link', 'name', 'code_out', 'cur_name', 
    'previousaum', 'currentaum', 'cashflow'
]
df = df.loc[
    df['date'] >= '2024-01-01', 
    cols
]
df = df.drop(
    df.loc[
        (df['currentaum'] == 0) & 
        (df['previousaum'] == 0)
    ].index
).reset_index(drop=True)

df = df.sort_values(
    ['name', 'code_out', 'date'], ignore_index=True
)

Идея: нужно прочитать файл с курсами валют и заджоинить с основной таблицей (нужны 2 столбцами с курсами на Т0 и Т-1)
- Прочитать файл с курсами
- Создать df с лагом по дате курсов
- Использовать melt, чтобы потом заджоинить с df

In [12]:
exchanges = pd.read_json('exchanges_profitability.json')
exchanges_lag = exchanges.shift()

exchanges = exchanges.reset_index().rename({'index': 'date'}, axis=1)
exchanges = exchanges.melt(id_vars='date', var_name='cur_name')

exchanges_lag = exchanges_lag.reset_index().rename({'index': 'date'}, axis=1)
exchanges_lag = exchanges_lag.melt(id_vars='date', var_name='cur_name')
exchanges_lag = exchanges_lag.rename({'value': 'value_t-1'}, axis=1)

df = pd.merge(
    df, exchanges, how='left',on=['date', 'cur_name']
)
df = pd.merge(
    df, exchanges_lag, how='left', on=['date', 'cur_name']
)

СЧА в таблицах представлено в рублях, нужно перевести в валюту стратегии
- Посчитать СЧА в валюте
- Посчитать дневную доходность в валюте (дох-ть пока что нужно оставить в формате 1+r, то есть не вычитаем единичку)
- Проставить дневную доходность равной 1 по строкам, в которых СЧА за пред. день равен 0 (это необходимо делать, поскольку иногда в cashflow отражается не совсем корректная инфа, из-за чего возникают беск. дох-ти)


In [17]:
df['previousaum_in_curr'] = df['previousaum'] / df['value_t-1']
df['currentaum_in_curr'] = df['currentaum'] / df['value']
df['cashflow_in_curr'] = df['cashflow'] / df['value']
df['daily_return'] = (df['currentaum_in_curr'] - df['cashflow_in_curr']) / df['previousaum_in_curr']

df = df.loc[
    (df['previousaum'] == 0)
    'daily_return'
] = 1

Подсчет кумулятивной доходности за за март
- Заслайсить df по датам
- Посчитать кумулятивную дох-ть по стратегиям
- Вычесть 1 из столбца дневной дох-ти
- Сортировать по 'name', 'code_out', 'date'

In [14]:
df1 = df.loc[
    (df['date'] <= '2024-03-29') & 
    (df['date'] >= '2024-03-01')
].copy()
df1['cumprod_daily_return'] = (df1.groupby(['name', 'code_out'])['daily_return'].cumprod() - 1)
df1['daily_return'] -= 1 
df1 = df1.sort_values(
    ['name', 'code_out', 'date'], ignore_index=True
)

Подсчет кумулятивной доходности с начала года
- Заслайсить df по датам
- Посчитать кумулятивную дох-ть по стратегиям
- Вычесть 1 из столбца дневной дох-ти
- Сортировать по 'name', 'code_out', 'date'

In [15]:
df2 = df.loc[
    (df['date'] <= '2024-03-29') & 
    (df['date'] >= '2024-01-01')
].copy()
df2['cumprod_daily_return'] = df2.groupby(['name', 'code_out'])['daily_return'].cumprod() - 1
df2['daily_return'] -= 1
df2 = df2.sort_values(
    ['name', 'code_out', 'date'], ignore_index=True
)

Вывести строки, где абсолютное значение дох-ти больше 15%

Это нужно для проверки корректности данных предоставляемые бэком, прдеполагается, что если дох-ть больше 15%, то со стороны бэка есть ошибка

In [16]:
df3 = df1[abs(df1['daily_return']) > 0.15].sort_values(['name', 'date'])
df3[['date', 'code_out', 'previousaum', 'currentaum', 'cashflow', 'daily_return']]

date  code_out  previousaum  currentaum  cashflow  daily_return
180  2024-03-07    BPIFMM         1.86        0.73       0.0     -0.603968
185  2024-03-12    BPIFMM         0.73        0.54       0.0     -0.259261
2966 2024-03-12  INDBONDQ     15298.80    12614.91       0.0     -0.175431
2973 2024-03-19  INDBONDQ     12614.91    10059.45       0.0     -0.202575
2975 2024-03-21  INDBONDQ     10059.45     7556.49       0.0     -0.248817
2976 2024-03-22  INDBONDQ      7556.49     2078.40       0.0     -0.724952
2979 2024-03-25  INDBONDQ      2078.40        0.00       0.0     -1.000000
3175 2024-03-22    BDSOV1    109169.53    85404.49       0.0     -0.211426
3908 2024-03-01     FBUSD         0.00        0.10       0.0           inf

Сохранить в эксель файл с 2 листами (первый и второй временной промежуток)

In [11]:
df2.to_excel('profitability.xlsx', index=False)

In [15]:
with pd.ExcelWriter('profitability.xlsx', mode='a', engine='openpyxl') as writer:
    df1.to_excel(writer, index=False, sheet_name='sheet2')